# En este notebook se leen fotogramas de un video y se obtiene la foto (tambien la guarda) y predicción del label box

In [1]:
import cv2
import time
import warnings; warnings.simplefilter('ignore')
import os

In [2]:
# Lanzamos nuestro otro notebook donde cargamos nuestro modelo y definimos las multiples funciones que usaremos
start_time = time.time()
%run FuncionesYolo.ipynb
print("--- %s seconds ---" % (time.time() - start_time))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
--- 40.326754093170166 seconds ---


In [3]:
import winsound
frequency = 3000  # Set Frequency To 3500 Hertz
duration = 350  # Set Duration To 250 ms == 1/4 second
frequency1=300   # Set Frequency To 300 Hertz
winsound.Beep(frequency, duration)
winsound.Beep(frequency1, duration)

In [31]:
#definimos el listado de precios de cada producto
precios=[1.23, 1.75, 2.22, 3.99, 4.00, 5.50, 1.99, 2.50, 0.99, 1.75]
precio={}
i=0
for e in labels:
    precio[e]=precios[i]
    i=i+1
precio    

{'ArrozCondis': 1.23,
 'ArrozExtra': 1.75,
 'HuevosFrescos': 2.22,
 'HuevosLibertad': 3.99,
 'KetchupCondis': 4.0,
 'KetchupHeinz': 5.5,
 'LecheAsturiana': 1.99,
 'LecheAto': 2.5,
 'MacarronesCondis': 0.99,
 'MacarronesGallo': 1.75}

In [32]:
#definimos una funcion que nos calcula el precio total de la lista dados dos diccionarios, el de precios individual, y el de la lista de la compra
def precioproductos(precio, lista_actual):
    suma=0
    for e in lista_actual:
        if lista_actual[e]!=0:
            valor= lista_actual[e]*precio[e]
            suma=suma+valor        
    
    return suma

In [30]:
d

{'ArrozCondis': 1,
 'ArrozExtra': 0,
 'HuevosFrescos': 0,
 'HuevosLibertad': 0,
 'KetchupCondis': 0,
 'KetchupHeinz': 1,
 'LecheAsturiana': 0,
 'LecheAto': 0,
 'MacarronesCondis': 0,
 'MacarronesGallo': 0}

In [29]:
precioproductos(precio, d)

6.73

In [4]:
# definimos los inputs fijos de nuestro modelo (tamaño de la fotografia, anchors y el threshhold):
input_w, input_h = 128, 128
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [[10,13,  16,30,  33,23],[30,61, 62,45,  59,119],[116,90,  156,198,  373,326]]
class_threshold = 0.85
c= 15 # los fotograms que queremos queignore antes seleccionar el siguiente. 30 equivale a avanzar 1 segundo. 15 equivale a 0,5 segundos
eliminamos = True #Si marcamos True, las imagenes seran eliminadas al instante. Las tenemos que crear temporalmente por el modelo.

In [36]:
d={}
for e in labels:
    d[e]=0
start_time = time.time()

#Importamos nuestro video
cap = cv2.VideoCapture('IMG_3777.MOV')
#cap = cv2.VideoCapture(2)
count = 0
l=[]
z=0
b=0
a=0
comp=0
reto=0
resultado=0
s=0

while cap.isOpened():
    ret, frame = cap.read() #Accedemos a cada frame

    if ret:
        #start_time1 = time.time()
        name='PruebaNotebook/frame'+str(count)+'.jpg' #Definimos donde guardamos nuestras fotografias
        cv2.imwrite(name, frame)
        #print(name + ' creado')
        image, image_w, image_h = load_image_pixels(name, (net_w, net_w)) #Obtenemos nuestas imagenes con el formato deseado
        yolos = yolov3.predict(image) #Hacemos la predicción de nuestro modelo
        boxes = list()
        for i in range(len(yolos)):
            boxes += decode_netout(yolos[i][0], anchors[i], obj_thresh,  net_h, net_w) #decodificamos el outoput de la red
        correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w) #Corregimos el tamaño de los bounding boxes
        do_nms(boxes, nms_thresh) #Eliminamos las non-maximal boxes
        # get the details of the detected objects
        v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold) #Obtenemos los detalles del objeto detectado
        
        
        
        if len(v_boxes) == 0:
            z=z+1
            print("Hemos detectado el fotograma numero " + str(s)+". No hay ningun productos detectado")
            if z>=2:
                z=0
                comp=0
                reto=0
                a=0
                b=0
                

                l.append(resultado)
                if resultado!=0:
                    if resultado[1]=="entrando":
                        d[resultado[0]] = d.get(resultado[0], 0) + 1
                        print("Validamos. Añadimos " +str(resultado[0]) + " a la lista de la compra")
                        winsound.Beep(frequency, duration)
                       
                    if resultado[1] == "saliendo":
                        d[resultado[0]] = d.get(resultado[0], 0) - 1
                        print("Validamos. Quitamos " +str(resultado[0]) + " a la lista de la compra")
                        winsound.Beep(frequency1, duration)
                    print("\nLa lista de la compra actual es:")
                    print(d)
                    print("El precio total actual de la compra es de", precioproductos(precio, d),"€")
                    print("\n")
                
               
                resultado=0
            s=s+1
        for i in range(len(v_boxes)):
            #[v_labels[i], v_boxes[i].xmin]
            if v_labels[i] == a:
                if z>=3:
                    z=0

                dif= v_boxes[i].xmin - b
                if dif >0:
                    comp= comp +1
                    print("Hemos detectado el fotograma numero " + str(s)+". Parece ser que: " + str(a) + " esta entrando")
                    print(comp)
                    if comp>=2:
                        resultado= [v_labels[i], "entrando"]
                        z=0
                    if comp>=2 and reto>=2:
                        resultado=0
                if dif<0:
                    reto= reto +1
                    print("Hemos detectado el fotograma numero " + str(s)+". Parece ser que: " + str(a) + " esta saliendo")
                    if reto>=2:
                        resultado= [v_labels[i], "saliendo"]
                        z=0
                    if comp>=2 and reto>=2:
                        resultado=0
                a, b = v_labels[i], v_boxes[i].xmin
                

            else:
                a, b = v_labels[i], v_boxes[i].xmin
                print("Hemos detectado el fotograma numero " + str(s)+". Hemos detectado un nuevo producto: " + str(a))

            s=s+1

            
            
            
            
        #draw_boxes(name, v_boxes, v_labels, v_scores) #Dibujamos nuestro fotografia con el objeto labelizado       
    
        count += c # Seleccionamos cuanto queremos que avance para seleccionar otro frame. Con 30 fps, avancamos 1 segundo
        cap.set(1, count)
        
        if eliminamos == True: #Marcamos si queremos eliminar o no las imagenes creadas.
            os.remove(name)
        
        if (count/30)%10 == 0:
            print("\n \n Estamos en el fotograma "+str(count)+" Y hemos estudiado un total de " +str(s)+"fotogramas")
            print("Hemos estudiado " + str(count/30) + " segundos de video \n \n")
        

    else:
        cap.release()
        break

print(l)
print(d)
print("El precio total actual de la compra es de", precioproductos(precio, d),"€")
print("--- %s seconds ---" % (time.time() - start_time))

Hemos detectado el fotograma numero 0. No hay ningun productos detectado
Hemos detectado el fotograma numero 1. No hay ningun productos detectado
Hemos detectado el fotograma numero 2. No hay ningun productos detectado
Hemos detectado el fotograma numero 3. Hemos detectado un nuevo producto: ArrozExtra
Hemos detectado el fotograma numero 4. Parece ser que: ArrozExtra esta entrando
1
Hemos detectado el fotograma numero 5. Parece ser que: ArrozExtra esta entrando
2
Hemos detectado el fotograma numero 6. Parece ser que: ArrozExtra esta entrando
3
Hemos detectado el fotograma numero 7. No hay ningun productos detectado
Hemos detectado el fotograma numero 8. No hay ningun productos detectado
Validamos. Añadimos ArrozExtra a la lista de la compra

La lista de la compra actual es:
{'ArrozCondis': 0, 'ArrozExtra': 1, 'HuevosFrescos': 0, 'HuevosLibertad': 0, 'KetchupCondis': 0, 'KetchupHeinz': 0, 'LecheAsturiana': 0, 'LecheAto': 0, 'MacarronesCondis': 0, 'MacarronesGallo': 0}
El precio total ac

In [6]:
d

{'ArrozCondis': 0,
 'ArrozExtra': 0,
 'HuevosFrescos': 0,
 'HuevosLibertad': 0,
 'KetchupCondis': 0,
 'KetchupHeinz': 0,
 'LecheAsturiana': 0,
 'LecheAto': 0,
 'MacarronesCondis': 0,
 'MacarronesGallo': 0}

In [7]:
l

[0,
 0,
 0,
 0,
 ['ArrozExtra', 'entrando'],
 0,
 0,
 0,
 ['ArrozExtra', 'saliendo'],
 0,
 0,
 0]